ДЗ 6. Metric model (regression)
Для датасету https://archive.ics.uci.edu/ml/datasets/Abalone побудувати метричну модель регресії
Таргет - вік (кількість кілець)
Додатково: класифікація, таргет - стать (infant, male, female)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, LabelEncoder

In [2]:
df = pd.read_csv('hw-6/abalone/abalone.data')
df.columns = ['Sex', 'Length', 'Diam', 'Height', 'Whole', 'Shucked', 'Viscera', 'Shell', 'Rings']
# Rings - is Target
df

In [3]:
df.dtypes


In [4]:
# Визначаємо категоріальні і числові фічі
num_features = ['Length', 'Diam', 'Height', 'Whole', 'Shucked', 'Viscera', 'Shell']
cat_features = ['Sex']

In [5]:
# Виводимо target окремо 
X_reg = df.drop(['Rings'], axis=1)
y_reg = df['Rings']

In [6]:
# Енкодимо категоріальні фічі та масштабуємо числові
onehot_encoder = OneHotEncoder()
cat_encoded = onehot_encoder.fit_transform(X_reg[cat_features]).toarray()
cat_encoded_df = pd.DataFrame(cat_encoded, columns=onehot_encoder.get_feature_names_out(cat_features))
scaler = StandardScaler()
num_scaled = scaler.fit_transform(X_reg[num_features])
num_scaled_df = pd.DataFrame(num_scaled, columns=num_features)

# Об'єднання закодованих категоріальних і масштабованих числових колонок
X_reg = pd.concat([cat_encoded_df, num_scaled_df], axis=1)

# Виведення результату
X_reg

In [7]:
# Розподіл даних на тренувальну та тестову вибірки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

In [8]:
# Побудова моделі регресії
reg_model = LinearRegression()
reg_model.fit(X_train_reg, y_train_reg)
y_pred_reg = reg_model.predict(X_test_reg)

# Оцінка моделі регресії
mse = mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)
mae = mean_absolute_error(y_test_reg, y_pred_reg)
rmse = mean_squared_error(y_test_reg, y_pred_reg)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

In [9]:
# Класифікація статі
X_clf = df.drop(['Sex', 'Rings'], axis=1)
y_clf = df['Sex']

# Кодування цільової змінної
label_encoder = LabelEncoder()
y_clf = label_encoder.fit_transform(y_clf)

# Масштабуємо числові, оскільки вік ніяк не впливає на стать то ми робимо це для всіх
scaler = StandardScaler()
X_clf = scaler.fit_transform(X_clf)

# Розподіл даних на тренувальну та тестову вибірки
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

# Побудова моделі класифікації
clf_model = LogisticRegression(max_iter=10000)
clf_model.fit(X_train_clf, y_train_clf)
y_pred_clf = clf_model.predict(X_test_clf)

# Оцінка моделі класифікації
accuracy = accuracy_score(y_test_clf, y_pred_clf)
report = classification_report(y_test_clf, y_pred_clf, target_names=label_encoder.classes_)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

In [10]:
# Результати не дуже тому вирішив глянути на фічі ще раз 

# Compute the correlation matrix
corr_matrix = df[num_features].corr()
print(corr_matrix)
# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [11]:
sns.pairplot(df, diag_kind='kde', plot_kws={'alpha':0.5})
plt.title('Pair Plot of Features')
plt.show()

In [12]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, palette='Set2')
plt.title('Box Plot of Features')
plt.xticks(rotation=90)
plt.show()

In [13]:
for column in df.columns:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.show()

In [14]:
# Проба побудови моделі з тюнінгом гіперпараметрів за допомогод GridSearchCv
# Define the ElasticNet model
elastic_net = ElasticNet()

# Define the hyperparameters grid
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization strength
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]   # Mix ratio between L1 and L2 penalty
}

# Setup the GridSearchCV
grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=5, scoring='r2')

# Fit the model
grid_search.fit(X_train_reg, y_train_reg)

# Best parameters found by GridSearchCV
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred_reg = best_model.predict(X_test_reg)

# Evaluate the model
mse = mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)
mae = mean_absolute_error(y_test_reg, y_pred_reg)
rmse = mean_squared_error(y_test_reg, y_pred_reg)

print("ElasticNet Regression with GridSearchCV:")
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

In [15]:
# Результат трохи кращий але все ще не дуже 
# Перевіряю чи можуть Polynominal Features дати кращі результати
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_reg)
X_test_poly = poly.transform(X_test_reg)

reg_model_poly = LinearRegression()
reg_model_poly.fit(X_train_poly, y_train_reg)
y_pred_poly = reg_model_poly.predict(X_test_poly)

r2_poly = r2_score(y_test_reg, y_pred_poly)
print(f'R^2 Score with Polynomial Features: {r2_poly}')

In [16]:
# Отже найбільший приріст дає використання Polynomial Features
# Загалом думаю результат можна покращити якщо подумати як вирішити питання залежних фіч типу висоти і ширини